### Introduction

In [2]:
import pandas as pd

In [3]:
week1 = pd.read_csv('Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('Restaurant - Week 2 Sales.csv') # connects customer and foods tables together through ID cols
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [6]:
customers = pd.read_csv('Restaurant - Customers.csv') # lists the customers
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [7]:
foods = pd.read_csv('Restaurant - Foods.csv') # lists the foods
foods.head()

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49


### The pd.concat() Method

Join 2 dfs together

In [8]:
pd.concat(objs = [week1, week2]) # week1 df is added first, Total rows 250+250, Index shows 250 as old indexes still used

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [9]:
pd.concat(objs = [week1, week2], ignore_index = True) # creates a new index to join the dfs together (500)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [11]:
week1.append(week2, ignore_index = True) # same thing but this will be deprecated soon

C:\Users\apati\AppData\Local\Temp\ipykernel_26424\4102608481.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  week1.append(week2, ignore_index = True) # This will be deprecated soon


,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [13]:
sales = pd.concat(objs = [week1, week2], keys = ['Week1', 'Week2']) # setting a multiIndex in concat() using keys param
sales

# keys act as the outer identifier to the df, gives a unique idnetifier without disturbing the original indexes of the dfs
# keys and objs should be same length

Customer ID  Food ID
Week1 0            537        9
      1             97        4
      2            658        1
      3            202        2
      4            155        9
...                ...      ...
Week2 245          783       10
      246          556       10
      247          547        9
      248          252        9
      249          249        6

[500 rows x 2 columns]

In [18]:
# Accessing multiIndex

sales.loc[('Week1', )] # All rows in week1 from resultant df
sales.loc[('Week1', 240)] # 240th row from week1

sales.loc[('Week1', 240), 'Customer ID'] # getting specific col from 240th row from week1
sales.loc[('Week2', 240), ['Customer ID', 'Food ID']] # getting a list of cols fro same entry

Customer ID    734
Food ID          1
Name: (Week2, 240), dtype: int64

### Inner Joins

In [37]:
week1 = pd.read_csv('Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('Restaurant - Customers.csv')
foods = pd.read_csv('Restaurant - Foods.csv')

In [20]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [21]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [23]:
# Will account for customers that visited in both week1 and week2(joins through common entries) -> Inner join

week1.merge(week2, how = 'inner', on = 'Customer ID', suffixes = [' - week1', ' -week2']) # inner join across 1 col
# week1 is the left table and weeK2 is the right table
# Sometimes multiple entries for ID show up, check below

,Customer ID,Food ID - week1,Food ID -week2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


In [25]:
week1[week1['Customer ID'] == 155]

,Customer ID,Food ID
4,155,9
17,155,1


In [26]:
week2[week2['Customer ID'] == 155] # As mentioned above, reason for multiple 'Customer ID' entries.

,Customer ID,Food ID
208,155,3


In [27]:
week1.merge(week2, how = 'inner', on = ['Customer ID', 'Food ID']) # inner join across multiple cols
# merges only rows from tables that have same 'Customer ID' and 'Food ID',
# i.e A.Customer_id = B.Customer_id & A.Food_id = B.Food_id

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


In [29]:
# Explanation of the duplicates in the above table

week1[week1['Customer ID'] == 578]

,Customer ID,Food ID
224,578,5


In [30]:
week2[week2['Customer ID'] == 578]

,Customer ID,Food ID
29,578,5
189,578,5


### Outer Joins

In [34]:
merged = week1.merge(week2, how = 'outer', on = 'Customer ID', suffixes = [' - Week 1',' - Week 2' ], indicator = True)

# combines all entries from both dfs, when we have a match it is converted to 1 row else pandas uses NaN for missing entry
# indicator param shows which table the data is coming from

In [35]:
merged['_merge'].value_counts() # useful

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

In [36]:
mask = merged['_merge'].isin(['left_only', 'right_only'])
merged[mask]
# removes the common entries between table 1 and table 2 (ID that shopped in both weeks, 
# keeps only ID's with either week1 or week2 value.

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


In [37]:
len(merged[mask])

392

### Left Joins

In [40]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [41]:
foods.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [42]:
week1.merge(foods, how = 'left', on = 'Food ID') # does a left join with week1 as left table and foods as the right table
# Useful when we have 1 main table and other is used as a reference to that

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99
...,...,...,...,...
245,413,9,Donut,0.99
246,926,6,Pasta,13.99
247,134,3,Taco,2.99
248,396,6,Pasta,13.99


In [43]:
week1.merge(foods, how = 'left', on = 'Food ID', sort = True) # sorts by on column

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
...,...,...,...,...
245,809,10,Drink,1.75
246,584,10,Drink,1.75
247,274,10,Drink,1.75
248,151,10,Drink,1.75


### The left_on and right_on Paramters

In [4]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [7]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [14]:
week2 = week2.merge(customers, how = 'left', left_on = 'Customer ID', right_on = 'ID', sort = True) #.drop('ID', axis = 'columns')
# left join, 2 cols that have different names from 2 tables'Customer ID' and 'ID'). Then we drop 'ID'.

In [15]:
week2

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,8,Frances,Adams,Female,Dabshots,Developer III
1,13,2,13,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,21,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,24,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,27,Jessica,Bennett,Female,Twitternation,Account Executive
...,...,...,...,...,...,...,...,...
245,968,4,968,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV
246,969,3,969,Marie,Ortiz,Female,Meejo,Research Associate
247,977,7,977,Cynthia,Dixon,Female,Skalith,Automation Specialist II
248,994,2,994,Clarence,Morgan,Male,Edgewire,Geologist II


### Merging by Indexes with the left_index and right_index Parameters

In [17]:
week1 = pd.read_csv('Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('Restaurant - Customers.csv', index_col = 'ID') # changing the index in customers table
foods = pd.read_csv('Restaurant - Foods.csv', index_col = 'Food ID')

In [20]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [18]:
foods.head(3)

,Food Item,Price
Food ID,,
1,Sushi,3.99
2,Burrito,9.99
3,Taco,2.99


In [19]:
customers.head(3)

,First Name,Last Name,Gender,Company,Occupation
ID,,,,,
1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
3,Roger,Black,Male,Tagfeed,Account Executive


In [26]:
sales = week1.merge(customers, how = 'left', left_on = 'Customer ID', right_index = True)
sales
# left join b/w the 'Customer ID' col from week1 table and Index(ID) from customers table -> use right_index param 
# duplicate rows for matching dont show when matching by index

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer


In [27]:
sales.merge(foods, how = 'left', left_on = 'Food ID', right_index = True)#merging the left table col with right table index

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III,Burrito,9.99
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III,Donut,0.99
...,...,...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer,Donut,0.99
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant,Pasta,13.99
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor,Taco,2.99
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer,Pasta,13.99


In [28]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [30]:
week1.merge(week2, how = 'left', left_index = True, right_index = True, suffixes = [' - Week1', ' - Week2']) 
# merging based on indexes for both tables (week1 and week2)

,Customer ID - Week1,Food ID - Week1,Customer ID - Week2,Food ID - Week2
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
...,...,...,...,...
245,413,9,783,10
246,926,6,556,10
247,134,3,547,9
248,396,6,252,9


### The .join() Method

In [31]:
satisfaction = pd.read_csv('Restaurant - Week 1 Satisfaction.csv')

In [32]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [33]:
satisfaction.head(3)

,Satisfaction Rating
0,2
1,7
2,3


In [34]:
week1.join(satisfaction).head() # joins the tables based on the indexes. Only works when both dfs have the same indexes.

# can be done using .merge() as well -> left_index = True & right_index = True

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


### The pd.merge() Method

In [35]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [38]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [40]:
pd.merge(week1, customers, how = 'left', left_on = 'Customer ID', right_on = 'ID').head(5) #just an alt syntax-> pd.merge()

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,537,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,97,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,658,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,202,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,155,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
